In [1]:
import re

In [2]:
with open('data/post.sql', 'r') as file:
    f = file.read()

In [71]:
def match(t):
    if t == 'int' or t == 'serial':
        return 'number'
    if t == 'char' or t == 'varchar':
        return 'string'
    if t == 'date' or t == 'timestamp':
        return 'Date'
    if t == 'boolean':
        return 'boolean'
    return ValueError('Unknown type')

In [115]:
entity = {}
for m in re.findall(r'CREATE TABLE .+ [(]\n[\s\w,\-()?]+\);', f):
    data = """import { IsNotEmpty, IsNumber, IsString, IsDate, IsBoolean } from "class-validator";
import { Column, Entity, PrimaryGeneratedColumn } from "typeorm";

"""
    table = m[13:-3].split(' (\n')
    name = table[0].title().replace('_', '').replace('report', 'Report')
    columns = table[1].replace(',', '').replace('    ', '').split('\n')
    data += f"@Entity()\nexport class {name} " + '{\n'
    data += f"""\n    @PrimaryGeneratedColumn()
    @IsNumber()
    @IsNotEmpty()
    id: number;\n"""
    for c in columns[1:]:
        n = c.split(' ')[0].lower()
        t = match(c.split(' ')[1].split('(')[0])
        data += f"\n    @Column()\n    @Is{t.title()}()\n"
        data += f"    {n}: {t};\n"
    data += '}'
    entity[name] = data

In [130]:
print(entity['InstitutionIndividual'])

import { IsNotEmpty, IsNumber, IsString, IsDate, IsBoolean } from "class-validator";
import { Column, Entity, PrimaryGeneratedColumn } from "typeorm";

@Entity()
export class InstitutionIndividual {

    @PrimaryGeneratedColumn()
    @IsNumber()
    @IsNotEmpty()
    id: number;

    @Column()
    @IsNumber()
    reportid: number;

    @Column()
    @IsNumber()
    individualid: number;
}


In [124]:
_ = [print(i) for i in entity.keys()]

Users
Contact
Address
Individual
Business
OtherReport
OtherIndividual
OtherBusiness
LaunderingReport
LaunderingIndividual
LaunderingBusiness
InstitutionReport
InstitutionIndividual
InstitutionBusiness
